## This script solves the cell cycle model for a WT cell and cells with a series of Myc mutations. Ref: figure 1a and figure S2

In [1]:
using DifferentialEquations 
using DataFrames
using Plots
using CSV
using Sundials
using JLD2
using OrdinaryDiffEq
using DelimitedFiles
using BSON
using FileIO

In [2]:
function condition(cellcycle,t,integrator) # Event when event_f(u,t) == 0
      numerator= cellcycle[indexOfRb]+cellcycle[indexOfE2FRb]+cellcycle[indexOfpE2FRb];  #ACTIVE RB
      denominator = numerator+cellcycle[indexOfHypoPRb]; #TOTAL RB

      #GM growth (biosynthesis of ribosomes and all necessary machinery)
      #previous iterations had numerator/denominator<0.8
      #diffEq.jl requires a function that hits 0
      (numerator/denominator)-0.8
end

function affect!(integrator)
  print("r31 event to 0\n")
  #update r31Switch to 0
  integrator.u[indexOfSwitch] = 0
end

function affectNeg!(integrator)
  print("r31 event to 1\n")
  #update r31Switch to 1
  integrator.u[indexOfSwitch] = 1
end

function conditionCdh(cellcycle,t,integrator) # Event when event_f(u,t) == 0

      #triggers when Cdh1 crosses 0.2 in the positive direction
      cellcycle[indexOfCdh1]-0.2
end

## all equal function
function allequal(obj)
    local x
    isfirst = true
    for i in obj
        if isfirst
            x = i
            isfirst = false
        else
            isequal(x, i) || return false
        end
    end
    return true
end

function affectCdh!(integrator)

  #triggers when Cdh1 crosses 0.2 in the positive direction
  #update Mass to 0.5* mass

   integrator.u[indexOfMass] = 0.5*integrator.u[indexOfMass]
#   #Gm to 0.5 * Gm
   integrator.u[indexOfGM]=0.5*integrator.u[indexOfGM]
# end
# end
  if integrator.u[indexOfMass]<0.5
    #print("new switch 0 during division\n")
    integrator.u[indexNewSwitch]=0
  else
    #print("new switch 1 during division\n")
    integrator.u[indexNewSwitch]=1
  end
 
    #store the latest peak mass in the last spot and shuffle every entry down one.
    integrator.u[indexOfMassTracker5]=integrator.u[indexOfMassTracker4]
    integrator.u[indexOfMassTracker4]=integrator.u[indexOfMassTracker3]
    integrator.u[indexOfMassTracker3]=integrator.u[indexOfMassTracker2]
    integrator.u[indexOfMassTracker2]=integrator.u[indexOfMassTracker1]
    integrator.u[indexOfMassTracker1]=round(integrator.u[indexOfMass], digits=3)
    integrator.u[indexOfMassTrackerTime5]=integrator.u[indexOfMassTrackerTime4]
    integrator.u[indexOfMassTrackerTime4]=integrator.u[indexOfMassTrackerTime3]
    integrator.u[indexOfMassTrackerTime3]=integrator.u[indexOfMassTrackerTime2]
    integrator.u[indexOfMassTrackerTime2]=integrator.u[indexOfMassTrackerTime1]
    integrator.u[indexOfMassTrackerTime1]=integrator.t
    lastPeaks=[integrator.u[indexOfMassTracker5] integrator.u[indexOfMassTracker4] integrator.u[indexOfMassTracker3] integrator.u[indexOfMassTracker2] integrator.u[indexOfMassTracker1]]
    if allequal(lastPeaks)
        
       terminate!(integrator)
    end

 print("division event\n")
end

## extra functions to stop the cell cycling when mass drops below 0.5

function conditionMass(cellcycle,t,integrator) # Event when event_f(u,t) == 0
      #triggers when mass crosses 0.5
      cellcycle[indexOfMass]-0.5
end

function affectMass!(integrator)
    integrator.u[indexNewSwitch] = 1
    print("new switch 1\n")
end

function affectMassNeg!(integrator)
    integrator.u[indexNewSwitch] = 0
    print("new switch 0\n")
end

affectMassNeg! (generic function with 1 method)

In [3]:
#use the callbacks described above to trigger events
cb = ContinuousCallback(condition,affect!,affectNeg!)
cb2 = ContinuousCallback(conditionCdh,affectCdh!,nothing)
cb3 = ContinuousCallback(conditionMass,affectMass!,affectMassNeg!)
cbs=CallbackSet(cb,cb2,cb3)

CallbackSet{Tuple{ContinuousCallback{typeof(condition), typeof(affect!), typeof(affectNeg!), typeof(SciMLBase.INITIALIZE_DEFAULT), typeof(SciMLBase.FINALIZE_DEFAULT), Float64, Int64, Rational{Int64}, Nothing, Int64}, ContinuousCallback{typeof(conditionCdh), typeof(affectCdh!), Nothing, typeof(SciMLBase.INITIALIZE_DEFAULT), typeof(SciMLBase.FINALIZE_DEFAULT), Float64, Int64, Rational{Int64}, Nothing, Int64}, ContinuousCallback{typeof(conditionMass), typeof(affectMass!), typeof(affectMassNeg!), typeof(SciMLBase.INITIALIZE_DEFAULT), typeof(SciMLBase.FINALIZE_DEFAULT), Float64, Int64, Rational{Int64}, Nothing, Int64}}, Tuple{}}((ContinuousCallback{typeof(condition), typeof(affect!), typeof(affectNeg!), typeof(SciMLBase.INITIALIZE_DEFAULT), typeof(SciMLBase.FINALIZE_DEFAULT), Float64, Int64, Rational{Int64}, Nothing, Int64}(condition, affect!, affectNeg!, SciMLBase.INITIALIZE_DEFAULT, SciMLBase.FINALIZE_DEFAULT, nothing, SciMLBase.LeftRootFind, 10, Bool[1, 1], 1, 2.220446049250313e-15, 0, 1

In [4]:
function initConditionsCellCycle(y0,syms,myc)
   #units: M 
   y0[findfirst(isequal("ERG"),syms)]=0.0121809 
   y0[findfirst(isequal("p27_cycA_Cdk2"),syms)]=0.0356927
   y0[findfirst(isequal("p27"),syms)]=0.00922806
   y0[findfirst(isequal("Cdc20"),syms)]=0.00220177 
   y0[findfirst(isequal("p27_cycE_Cdk2"),syms)]=0.000542587
   y0[findfirst(isequal("cycE"),syms)]=0.0229112 
   y0[findfirst(isequal("cycA"),syms)]=1.4094 
   y0[findfirst(isequal("cycB"),syms)]=2.72898
   y0[findfirst(isequal("p27_cycD_Cdk2"),syms)]=0.010976 
   y0[findfirst(isequal("cycD"),syms)]=0.43929 
   y0[findfirst(isequal("Cdh1"),syms)]=0.000653278 
   y0[findfirst(isequal("DRG"),syms)]=0.900533
   y0[findfirst(isequal("PPX"),syms)]=1.0
   y0[findfirst(isequal("IEP"),syms)]=0.154655
   y0[findfirst(isequal("Cdc20t"),syms)]=2.36733
   y0[findfirst(isequal("E2F_Rb"),syms)]=0.00478911
   y0[findfirst(isequal("E2F"),syms)]=0.989986
   y0[findfirst(isequal("HypoP_Rb"),syms)]=9.97574
   y0[findfirst(isequal("pE2F_Rb"),syms)]=0.0192822
   y0[findfirst(isequal("pE2F"),syms)]=3.98594
   y0[findfirst(isequal("GM"),syms)]=1.35565
   y0[findfirst(isequal("r31switch"),syms)]=1
   y0[findfirst(isequal("cMyc"),syms)]=myc
   y0[findfirst(isequal("Mass"),syms)]=1
   y0[findfirst(isequal("newSwitch"),syms)]=1
   y0[findfirst(isequal("Rb"),syms)]=0
   y0[findfirst(isequal("cMyct"),syms)]=40
   return y0
end

initConditionsCellCycle (generic function with 1 method)

In [5]:
include("variableNames.jl")
global indexCdh1=findfirst(x->"Cdh1"==x,syms)
global indexMass=findfirst(x->"Mass"==x,syms)
global indexcycE=findfirst(x->"cycE"==x,syms)
global indexcycA=findfirst(x->"cycA"==x,syms)
global indexcycB=findfirst(x->"cycB"==x,syms)
global indexcycD=findfirst(x->"cycD"==x,syms)
global indexIEP=findfirst(x->"IEP"==x,syms)
global indexnewSwitch=findfirst(x->"newSwitch"==x,syms)
global indexmyc=findfirst(x->"cMyc"==x,syms)
global indexmyct=findfirst(x->"cMyct"==x,syms)
global indexp27=findfirst(x->"p27_cycE_Cdk2"==x,syms)

5

### This function calculates the time point in the solution of the model at which transitions between different stages of the cell cycle occur.

In [6]:
### NEW FUNCTION

function Get_CC_transitions(sol_SS, time_stamp_SS)
  Cdh1_col = columnindex(sol_SS, :Cdh1)
  cycA_col = columnindex(sol_SS, :cycA)
  cycB_col = columnindex(sol_SS, :cycB)
  cycE_col = columnindex(sol_SS, :cycE)
  Mass_col = columnindex(sol_SS, :Mass)

  sol_SS_cc = sol_SS[:, [Cdh1_col,cycA_col,cycB_col,cycE_col,Mass_col]]

  # G1 -> S boundary when cycB is > 0 (or close to 0)cycB_peak_df = convert_colname(max_cycB[2])
  # S to G2 when CycE is gone but before peak in CycA
  # G2 to M at peak of CycB before rapid decrease in CycB, CycA should already be decreasing and about half way gone.

  SS_zero = convert(Int64, round(time_stamp_SS[1], digits=0))

  ##G1/S
  max_Cdh1 = findmax(sol_SS_cc[:,1])
  submax_Cdh1 = (max_Cdh1[1]/100)*95
  submax_Cdh1 = findfirst(x->x>submax_Cdh1, sol_SS_cc[:,1])
  G1S_df = sol_SS_cc[submax_Cdh1:nrow(sol_SS_cc),:]
  col_Cdh1 = G1S_df[:,1]  
  half_max_Cdh1 = max_Cdh1[1]*0.5
  G1S=findfirst(x->x<half_max_Cdh1,col_Cdh1)
  G1S_t = convert(Int64, round(time_stamp_SS[G1S+submax_Cdh1],digits=0))
  G1S_t = G1S_t - SS_zero
    
  ## S/G2
  G1St = SS_zero+G1S_t
  G1St_TS = findfirst(x->x>G1St, time_stamp_SS)
  # get max value of cycA
  colcycA = sol_SS_cc[:,2]
  max_cycA = findmax(colcycA[G1St_TS:end])
  # get time when cycA is max - 2.5%
  submaxA = (max_cycA[1]/100)*97.5
  submaxA = findfirst(x->x>submaxA,colcycA)
  SG2_t = convert(Int64, round(time_stamp_SS[submaxA],digits=0))
  SG2_t = SG2_t - SS_zero

  ## G2/M
  SG2t = SS_zero+SG2_t
  SG2t_TS = findfirst(x->x>SG2t, time_stamp_SS)
  col_cycB = sol_SS_cc[:,3]
  max_cycB = findmax(col_cycB[SG2t_TS:end])
  #submaxB = (max_cycB[1]/100)*97.5
  max_cycB = findfirst(x->x==max_cycB[1],col_cycB)
  G2M_time = convert(Int64, round(time_stamp_SS[max_cycB],digits=0))
  G2M_t = G2M_time - SS_zero

  ## M
  M_t = convert(Int64, round(time_stamp_SS[end],digits=0)) - SS_zero

  return(G1S_t,SG2_t,G2M_t,M_t)

end

Get_CC_transitions (generic function with 1 method)

In [7]:
global colnames=["G1S", "SG2", "G2M", "M"];

### Define the problem

In [8]:
### Files
reactionsFile="reactions_plus_myc3_NS_Mass2.csv"

## WT sim
parametersFile="parameters_plus_myc.csv"
## Myc overexpression
#parametersFile="moduleDefinitionFiles/cellCycle/parameters_plus_myc_mut.csv"

rateLawsFile="rateLaws_plus_myc3_NS_Mass2.csv"

### Run csv2julia
arguments=[reactionsFile, parametersFile, rateLawsFile, "odeCellCycle.jl", "scan"]
locationOfCSV2Julia="csv2model-multiscale.py"
cmd=`python3 $locationOfCSV2Julia $arguments`
run(cmd)

include("odeCellCycle.jl")
include("variableNames.jl")
include("scanIncludes.jl");


### Define variables
locationOfVariableNames="variableNames.jl"
include(locationOfVariableNames)
indexOfMass=findfirst(x->"Mass"==x,syms)
indexOfGM=findfirst(x->"GM"==x,syms)
indexNewSwitch=findfirst(x->"newSwitch"==x,syms)
indexOfRb=findfirst(x->"Rb"==x,syms)
indexOfE2FRb=findfirst(x->"E2F_Rb"==x,syms)
indexOfpE2FRb=findfirst(x->"pE2F_Rb"==x,syms)
indexOfHypoPRb=findfirst(x->"HypoP_Rb"==x,syms)
indexOfSwitch=findfirst(x->"r31switch"==x,syms)
indexOfCdh1=findfirst(x->"Cdh1"==x,syms)
indexOfMassTracker1=findfirst(x->"previousMass1"==x,syms)
indexOfMassTracker2=findfirst(x->"previousMass2"==x,syms)
indexOfMassTracker3=findfirst(x->"previousMass3"==x,syms)
indexOfMassTracker4=findfirst(x->"previousMass4"==x,syms)
indexOfMassTracker5=findfirst(x->"previousMass5"==x,syms)
indexOfMassTrackerTime1=findfirst(x->"previousMassTime1"==x,syms)
indexOfMassTrackerTime2=findfirst(x->"previousMassTime2"==x,syms)
indexOfMassTrackerTime3=findfirst(x->"previousMassTime3"==x,syms)
indexOfMassTrackerTime4=findfirst(x->"previousMassTime4"==x,syms)
indexOfMassTrackerTime5=findfirst(x->"previousMassTime5"==x,syms)

### Set solve conditions
syms=Symbol.(syms)

maxTimeSS=100000.0
maxTimeTC=24*60.0

DISincrease=0.000001

## set time to solve
TCLength=300*60
## set max attempts to find steady state
maximumAttemptsAtSS=20

odeFile=include("odeCellCycle.jl")
f=ODEFunction(odeFile,syms=syms)
y0=zeros(length(syms))
y0=initConditionsCellCycle(y0,String.(syms), 40)
global prob=ODEProblem(f,y0,(0.0,maxTimeSS))

scan
Running CSV2JuliaDiffEq with parameters left as a function call to paramFun(n), for all params. We will also create a paramFun.jl file that should be included and defines all parameters. If this is incorrect, please re-run with 5th argument set to 'inline' or 'param'
Opening rateLaws_plus_myc3_NS_Mass2.csv as rate law file
Opening parameters_plus_myc.csv as parameters file
Opening reactions_plus_myc3_NS_Mass2.csv as reactions file
parameters can now be modified by name.
example to modify k_binding 1.5 fold higher:
modify["k_binding"]=1.5


ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 100000.0)
u0: 37-element Vector{Float64}:
  0.0121809
  0.0356927
  0.00922806
  0.00220177
  0.000542587
  0.0229112
  1.4094
  2.72898
  0.010976
  0.43929
  0.000653278
  0.900533
  1.0
  ⋮
  0.0
 40.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

### Subset steady state

In [9]:
function subsetLC(sol)
    startOfSSIndex=sol[3]
    sol_SS = sol[1][startOfSSIndex:end]
    return(sol_SS)
end

subsetLC (generic function with 1 method)

### Solve

In [10]:
function solvesingle(prob, val)
    run(cmd)

    include("odeCellCycle.jl")
    include("variableNames.jl")
    include("scanIncludes.jl")

    modify["k1_myc"] = val
    
    sol = solve(prob,callback=cbs, abstol=1e-5,reltol=1e-3,saveat=1.0)

    ### get steady state - steady state = time between peak mass values plus division (when mass drops to zero)
    sol_df = DataFrame(sol', syms)
    time_stamp = sol.t
    sol_df[!, :time_stamp] = time_stamp

    indexOfLastCycleStart=findfirst(x->x>sol[indexOfMassTrackerTime2,end],sol.t)
    sol_SS = sol_df[indexOfLastCycleStart:end, :]

    # get time stamp
    time_stamp_SS = sol_SS[:, :time_stamp]

    ### Calculate times for cell cycle transitions
    colnames=["G1S", "SG2", "G2M", "M"]
    plot_times = DataFrame(Array(collect(Get_CC_transitions(sol_SS, time_stamp_SS))'), colnames)
    show(plot_times)
    
    return(sol, plot_times, indexOfLastCycleStart)
end

solvesingle (generic function with 1 method)

In [11]:
paramValsWTMyc=[1,1.5]
paramValsWTMyc2=[1,1.5,2,5,10,50,100]

7-element Vector{Float64}:
   1.0
   1.5
   2.0
   5.0
  10.0
  50.0
 100.0

In [12]:
function solveList(paramVals)
    solList=[]
    for i in 1:length(paramVals)
        soln = solvesingle(prob, paramVals[i])
        sol_LC = subsetLC(soln)
        # get limit cycle timestamp
        sol_LC.t .= sol_LC.t .- sol_LC.t[1]
        push!(solList, sol_LC)
    end
    return(solList)
end

solveList (generic function with 1 method)

In [13]:
solList_WT_Myc = solveList(paramValsWTMyc)

scan
Running CSV2JuliaDiffEq with parameters left as a function call to paramFun(n), for all params. We will also create a paramFun.jl file that should be included and defines all parameters. If this is incorrect, please re-run with 5th argument set to 'inline' or 'param'
Opening rateLaws_plus_myc3_NS_Mass2.csv as rate law file
Opening parameters_plus_myc.csv as parameters file
Opening reactions_plus_myc3_NS_Mass2.csv as reactions file
parameters can now be modified by name.
example to modify k_binding 1.5 fold higher:
modify["k_binding"]=1.5
division event
division event
division event
division event
division event
division event
division event
division event
division event
division event
division event
division event
division event
division event
division event
division event
division event
division event
division event
1×4 DataFrame
 Row │ G1S    SG2    G2M    M     
     │ Int64  Int64  Int64  Int64 
─────┼────────────────────────────
   1 │   303    587    688    730scan
Running C

2-element Vector{Any}:
 ODESolution{Float64, 2, Vector{Vector{Float64}}, Nothing, Nothing, Vector{Float64}, Vector{Vector{Vector{Float64}}}, ODEProblem{Vector{Float64}, Tuple{Float64, Float64}, true, SciMLBase.NullParameters, ODEFunction{true, SciMLBase.FullSpecialize, typeof(odeCellCycle), LinearAlgebra.UniformScaling{Bool}, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Vector{Symbol}, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED), Nothing, Nothing}, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}, SciMLBase.StandardODEProblem}, CompositeAlgorithm{Tuple{Tsit5{typeof(OrdinaryDiffEq.trivial_limiter!), typeof(OrdinaryDiffEq.trivial_limiter!), Static.False}, Rosenbrock23{10, false, LinearSolve.DefaultLinearSolver, typeof(OrdinaryDiffEq.DEFAULT_PRECS), Val{:forward}, true, nothing}}, OrdinaryDiffEq.AutoSwitchCache{Tsit5{typeof(OrdinaryDiffEq.trivial_limiter!), typeof(OrdinaryDiffEq.trivial_limiter!), Static.False}, Rosenbrock

In [14]:
function solveList2(paramVals)
    plot_times_df = DataFrame(G1S=Int64[], SG2=Int64[], G2M=Int64[], M=Int64[], mut=Float64[])
    for val in paramVals
        soln = solvesingle(prob, val)
        CC_times = soln[2]
        CC_times[!, :mut] = [val]
        append!(plot_times_df, CC_times)
    end
    return(plot_times_df)
end

solveList2 (generic function with 1 method)

In [15]:
solList_Myc1to100 = solveList2(paramValsWTMyc2)

scan
Running CSV2JuliaDiffEq with parameters left as a function call to paramFun(n), for all params. We will also create a paramFun.jl file that should be included and defines all parameters. If this is incorrect, please re-run with 5th argument set to 'inline' or 'param'
Opening rateLaws_plus_myc3_NS_Mass2.csv as rate law file
Opening parameters_plus_myc.csv as parameters file
Opening reactions_plus_myc3_NS_Mass2.csv as reactions file
parameters can now be modified by name.
example to modify k_binding 1.5 fold higher:
modify["k_binding"]=1.5
division event
division event
division event
division event
division event
division event
division event
division event
division event
division event
division event
division event
division event
division event
division event
division event
division event
division event
division event
1×4 DataFrame
 Row │ G1S    SG2    G2M    M     
     │ Int64  Int64  Int64  Int64 
─────┼────────────────────────────
   1 │   303    587    688    730scan
Running C

,G1S,SG2,G2M,M,mut
,Int64,Int64,Int64,Int64,Float64
1,303,587,688,730,1.0
2,295,579,680,721,1.5
3,292,576,676,718,2.0
4,289,574,673,715,5.0
5,289,573,673,715,10.0
6,288,573,673,714,50.0
7,289,573,673,715,100.0


In [16]:
# get length of each part of the cell cycle from the end times
function res_mod(df)
      for i in 1:nrow(df)
            df[i,4]=df[i,4] - df[i,3]
            df[i,3]=df[i,3] - df[i,2]
            df[i,2]=df[i,2] - df[i,1]
      end
      return(df)
end

res_mod (generic function with 1 method)

In [17]:
solList_Myc1to100c = deepcopy(solList_Myc1to100)

,G1S,SG2,G2M,M,mut
,Int64,Int64,Int64,Int64,Float64
1,303,587,688,730,1.0
2,295,579,680,721,1.5
3,292,576,676,718,2.0
4,289,574,673,715,5.0
5,289,573,673,715,10.0
6,288,573,673,714,50.0
7,289,573,673,715,100.0


In [18]:
solList_Myc1to100_mod = res_mod(solList_Myc1to100)

,G1S,SG2,G2M,M,mut
,Int64,Int64,Int64,Int64,Float64
1,303,284,101,42,1.0
2,295,284,101,41,1.5
3,292,284,100,42,2.0
4,289,285,99,42,5.0
5,289,284,100,42,10.0
6,288,285,100,41,50.0
7,289,284,100,42,100.0


In [19]:
solList_Myc1to100c

,G1S,SG2,G2M,M,mut
,Int64,Int64,Int64,Int64,Float64
1,303,587,688,730,1.0
2,295,579,680,721,1.5
3,292,576,676,718,2.0
4,289,574,673,715,5.0
5,289,573,673,715,10.0
6,288,573,673,714,50.0
7,289,573,673,715,100.0


### Plot

In [20]:
function plotSolSS(solSS, sol2SS, title)
    p1=plot(solSS, vars=[indexCdh1,indexcycE,indexcycA,indexcycB,indexcycD,indexMass],
          lc=[:green :yellow :purple :red :blue :orange],
          lw=2.5,
          xlabel="time (hours)",
        xlims=(0,800),
        xticks=(1:300:780, string.(0:5:13)),
          legend= :outerbottomright,
          #ylims=(0,5),
          )   
     plot!(p1,sol2SS, vars=[indexCdh1,indexcycE,indexcycA,indexcycB,indexcycD,indexMass],
          lc=[:green2 :yellow2 :purple2 :red2 :blue2 :orange2],
          linestyle=:dash,
          legend= :outerbottomright,
        xlabel="time (hours)"
          ) 
    vline!([309], color=:black, linestyle=:dash, lw=1, label="G1/S WT 309 mins")
    vline!([592], color=:black, linestyle=:dash, lw=1.5, label="S/G2 WT 592 mins")
    vline!([688], color=:black, linestyle=:dash, lw=2, label="G2/M WT 688 mins")
    vline!([730], color=:black, linestyle=:dash, lw=2.5, label="M WT 730 mins")
end

plotSolSS (generic function with 1 method)

In [21]:
plotly()

Plots.PlotlyBackend()

In [22]:
plotSolSS(solList_WT_Myc[1], solList_WT_Myc[2], "WT vs Myc (x1.5) cell cycle")

┌ Warning: To maintain consistency with solution indexing, keyword argument vars will be removed in a future version. Please use keyword argument idxs instead.
│   caller = ip:0x0
└ @ Core :-1


In [23]:
xt = ["1","1.5","2","5","10","50","100"]

7-element Vector{String}:
 "1"
 "1.5"
 "2"
 "5"
 "10"
 "50"
 "100"

In [24]:
plot(1:7, solList_Myc1to100_mod.G1S, ylim=(280,310), label="G1", xticks = (1:7,xt),linewidth=5,colour=:purple,
    xlab="CMYC multiplier", ylab = "time (mins)", size=(300,300),grid=false)
plot!(1:7, solList_Myc1to100_mod.G1S, seriestype=:scatter, label=false, colour=:purple, markersize=5, grid=false)

In [25]:
plot(1:7, solList_Myc1to100_mod.SG2, ylim=(270,300), label="S", xticks = (1:7,xt),linewidth=5,colour=:green,
    xlab="CMYC multiplier", ylab = "time (mins)", size=(300,300),grid=false)
plot!(1:7, solList_Myc1to100_mod.SG2, seriestype=:scatter, label=false, colour=:green, markersize=5)

In [26]:
plot(1:7, solList_Myc1to100_mod.G2M, ylim=(85,115), label="G2", xticks = (1:7,xt), linewidth=5,colour=:orange,
    xlab="CMYC multiplier", ylab = "time (mins)", size=(300,300),grid=false)
plot!(1:7, solList_Myc1to100_mod.G2M, seriestype=:scatter, label=false, colour=:orange,markersize=5)

In [27]:
plot(1:7, solList_Myc1to100_mod.M, ylim=(25,55), label="M", xticks = (1:7,xt),linewidth=5,colour=:blue,
    xlab="CMYC multiplier", ylab = "time (mins)",size=(300,300),grid=false)
plot!(1:7, solList_Myc1to100_mod.M, seriestype=:scatter, label=false, colour=:blue, markersize=5)

In [28]:
plot(solList_Myc1to100c.M, seriestype=:bar, orientation=:h, yflip=true, ylim = (0,8), 
    yticks=(1:7,xt),barwidth=0.5,
    colour=:grey, label=false, size=(900,300))
s=string.(collect(0:1:12))
plot!(xticks = (0:(60*1):(60*12),s))